In [31]:
import sys
import os
import wandb
sys.path.append('/u/home/koksal/organ-mesh-registration-and-property-prediction/')

import torch
from src.models.fsgn_model import MeshSeg
from src.data.organs_dataset import OrganMeshDataset
from torch_geometric.data import DataLoader
from tqdm import tqdm
from time import sleep
import mlflow
from src.models.baseline_model import GNN
import argparse
from sklearn.metrics import r2_score

#config = Args()
from src.models.train import build_dataset, build_network


In [32]:
experiment_dir = '/u/home/koksal/organ-mesh-registration-and-property-prediction/models/summer-field-27'
checkpoint_path ='classification_organ_left_kidney_enc_channels_512_best_testacc_0.48.pth'
checkpoint = torch.load(os.path.join(experiment_dir, checkpoint_path))
config_dict = checkpoint['config']

In [33]:
class Dict2Class(object):
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])

In [34]:
config = Dict2Class(config_dict)

In [28]:
#Data Loader
train_loader, test_loader = build_dataset(config)
#Network
net = build_network(config).to(config.device)

Left_kidney  train Dataset is created
Left_kidney  val Dataset is created
Left_kidney  test Dataset is created
Baseline Model is initialized


/u/home/koksal/.conda/envs/caghan4/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [29]:
net.load_state_dict(checkpoint['model'])


<All keys matched successfully>

DataBatch(x=[16302, 3], edge_index=[2, 65180], y=[4], num_nodes=16302, eid=[4], batch=[16302], ptr=[5])

In [44]:
batch = next(iter(train_loader)).cuda()
net.cuda()

GNN(
  (input_encoder): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): Identity()
  )
  (layers): ModuleList(
    (0): GCNConv(128, 512)
    (1): ReLU()
    (2): LayerNorm(512, mode=graph)
    (3): GCNConv(512, 512)
    (4): ReLU()
    (5): LayerNorm(512, mode=graph)
    (6): GCNConv(512, 512)
    (7): ReLU()
    (8): LayerNorm(512, mode=graph)
  )
  (pred_layer): Linear(512, 1, bias=True)
)

In [55]:
net.eval()
predictions = net(batch)

In [56]:
from sklearn.metrics import r2_score

In [65]:
from torchmetrics import R2Score
r2score = R2Score().to('cuda')
r2score(predictions.squeeze(1), batch.y)

tensor(-3., device='cuda:0', grad_fn=<SqueezeBackward0>)

In [68]:
predictions,batch.y

(tensor([[0.],
         [0.],
         [0.],
         [0.]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([1, 1, 1, 0], device='cuda:0'))

In [66]:
r2_score(predictions.cpu().detach().numpy(),batch.y.cpu())

0.0

In [48]:
batch.y.shape

torch.Size([4])